In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
import typhon as ty

import konrad


ty.plots.styles.use()

# Greogy method

In [ ]:
def calc_forcing_regression(
    co2_factor=2.0,
    co2_reference=348e-6,
    T_s=290.0,
    humidity=None,
    lapserate=None,
    ):
    """Calculate the effective radiative forcing based on two SST RCE runs."""
    phlev = konrad.utils.get_quadratic_pgrid(1000e2, 10, 128)
    atmosphere = konrad.atmosphere.Atmosphere(phlev)
    
    atmosphere["CO2"][:] = co2_reference # Set reference CO2 concentration
    
    # Calculate reference OLR.
    spinup = konrad.RCE(
        atmosphere,
        surface=konrad.surface.FixedTemperature(temperature=T_s),
        timestep='24h',  # Set timestep in model time.
        lapserate=lapserate,
        max_duration='150d',  # Set runtime.
    )
    spinup.run()  # Start the simulation.

    # Calculate OLR at perturbed atmospheric state.
    atmosphere["CO2"][:] *= co2_factor # double the CO2 concentration
    
    perturbed = konrad.RCE(
        atmosphere,
        surface=konrad.surface.SlabOcean(
            temperature=T_s,
            heat_sink=spinup.radiation["toa"][-1]
        ),
        timestep='12h',  # Set timestep in model time.
        humidity=spinup.humidity if humidity is None else humidity,
        lapserate=spinup.lapserate,
        max_duration='350d',  # Set runtime.
        outfile="perturbed.nc",
        writeevery="5d",
    )
    perturbed.run()  # Start the simulation.
    
    with netCDF4.Dataset("perturbed.nc", "r") as root:
        Ts = root["surface/temperature"][:]
        toa = root["radiation/toa"][:]
        heat_sink = root["surface/heat_sink"][-1]
    
    dTs = Ts - Ts[0]
    toa = toa - heat_sink
    is_adjusted = np.abs(dTs) > 0.3
    
    return np.polyfit(dTs[is_adjusted], toa[is_adjusted], deg=1)[1]

# Fixed SST approach

In [ ]:
def calc_effective_forcing(
    co2_factor=2.0,
    co2_reference=348e-6,
    T_s=290.0,
    humidity=None,
    lapserate=None,
    ):
    """Calculate the effective radiative forcing based on two SST RCE runs."""
    phlev = konrad.utils.get_quadratic_pgrid(1000e2, 10, 128)
    atmosphere = konrad.atmosphere.Atmosphere(phlev)
    
    atmosphere["CO2"][:] = co2_reference # Set reference CO2 concentration
    
    # Calculate reference OLR.
    spinup = konrad.RCE(
        atmosphere,
        surface=konrad.surface.FixedTemperature(temperature=T_s),
        timestep='24h',  # Set timestep in model time.
        lapserate=lapserate,
        max_duration='150d',  # Set runtime.
    )
    spinup.run()  # Start the simulation.

    # Calculate OLR at perturbed atmospheric state.
    atmosphere["CO2"][:] *= co2_factor # double the CO2 concentration
    
    perturbed = konrad.RCE(
        atmosphere,
        surface=spinup.surface,
        timestep='12h',  # Set timestep in model time.
        humidity=spinup.humidity if humidity is None else humidity,
        lapserate=spinup.lapserate,
        max_duration='150d',  # Set runtime.
    )
    perturbed.run()  # Start the simulation.
    
    return perturbed.radiation["toa"][-1] - spinup.radiation["toa"][-1]

# Compare approaches

In [ ]:
def plot_forcing_dependence(forcing_function):
    configurations = {
        "FixedRH": {"humidity": None, "lapserate": None},
        "FixedVMR": {"humidity": konrad.humidity.FixedVMR(), "lapserate": None},
        "FixedLR": {"humidity": konrad.humidity.FixedVMR(), "lapserate": konrad.lapserate.FixedLapseRate()},
    }

    co2_factors = [0.5, 2, 4, 8]

    fig, ax = plt.subplots()
    ax.axhline(0, lw=0.8, c="k")
    ax.axvline(1, lw=0.8, c="k")
    for label, kwargs in configurations.items():
        forcings = []
        for co2_factor in co2_factors:
            forcings.append(forcing_function(
                co2_factor=co2_factor,
                **kwargs,
            ))

        l_e = ax.plot(co2_factors, forcings, marker="D", label=label)
        print(label, forcings)

    ax.set_ylabel(r"$\Delta F$ / $\rm Wm^{-2}$")
    ax.set_xlabel(r"$ n \times \rm CO_2$")

    ax.legend(fontsize="small")
    ax.set_xscale("log")
    ax.set_xticklabels(co2_factors)
    ax.set_xticks(co2_factors)
    ax.set_xticks([], minor=True)

In [ ]:
plot_forcing_dependence(calc_forcing_regression)
plot_forcing_dependence(calc_effective_forcing)